<h1 align="center">VGG16 Image Recognition（CIFAR-1000 物件）</h1>

<hr>
<table>
    <tr>
    <td><img src="https://www.hibiscusgarden.com/wp-content/uploads/2017/12/pina.jpg" width="300"></td>
    <td><img src="https://cdn.shopify.com/s/files/1/0267/7948/4355/articles/88f44c1d5f09f8eb01b557ad014b743d_600x.jpg" width="300"></td>
    </tr>
</table>
<pre>
上圖取自
<a href="https://cdn.shopify.com/s/files/1/0267/7948/4355/articles/88f44c1d5f09f8eb01b557ad014b743d_600x.jpg">https://cdn.shopify.com/s/files/1/0267/7948/4355/articles/88f44c1d5f09f8eb01b557ad014b743d_600x.jpg</a>
<a href="https://www.hibiscusgarden.com/wp-content/uploads/2017/12/pina.jpg">https://www.hibiscusgarden.com/wp-content/uploads/2017/12/pina.jpg</a>
</pre>

<hr>
<h3>CIFAR-1000 class names（物體類別名稱）from ImageNet</h3>
<pre>
<a href="https://github.com/xmartlabs/caffeflow/blob/master/examples/imagenet/imagenet-classes.txt">imagenet_classes.txt</a>
pineapple (class index = 953)
banana (class index = 954)
</pre>

<hr>
<h3>載入 CIFAR-1000 物體類別名稱</h3>

In [ ]:
# 載入 CIFAR-1000 物體類別名稱

with open('imagenet_classes.txt', 'r') as fp:
    txt = fp.read()
fp.close()

class_names = txt.split('\n')

print(class_names[950:960])


<hr>
<h3>載入 VGG16 Model</h3>
<p>下載至個人根目錄下</p>

In [ ]:
# 載入 VGG16 Model

import torch

# 下載
vgg16 = torch.hub.load('pytorch/vision:v0.6.0', 'vgg16', pretrained=True)

# ANN 網路模型執行（原定義啟用）
vgg16.eval()

# 顯示定義
print(vgg16)


<hr>
<h3>其它 VGG 變形</h3>

In [ ]:
# vgg variants
# vgg = torch.hub.load('pytorch/vision:v0.6.0', 'vgg11', pretrained=True)
# vgg = torch.hub.load('pytorch/vision:v0.6.0', 'vgg11_bn', pretrained=True)
# vgg = torch.hub.load('pytorch/vision:v0.6.0', 'vgg13', pretrained=True)
# vgg = torch.hub.load('pytorch/vision:v0.6.0', 'vgg13_bn', pretrained=True)
# vgg = torch.hub.load('pytorch/vision:v0.6.0', 'vgg16', pretrained=True)
# vgg = torch.hub.load('pytorch/vision:v0.6.0', 'vgg16_bn', pretrained=True)
# vgg = torch.hub.load('pytorch/vision:v0.6.0', 'vgg19', pretrained=True)
# vgg = torch.hub.load('pytorch/vision:v0.6.0', 'vgg19_bn', pretrained=True)

<hr>
<h3>Image Pre-Processing 定義</h3>
<p>torchvision 工具模組的 transforms 功能預設使用 Image 工具模組的影像格式</p>
<pre>
影像轉為 Tensor 格式（shape: 3,224,224）
</pre>

In [ ]:
# Image Pre-Processing 定義

from torchvision import transforms

preprocess = transforms.Compose([
    transforms.Resize(256), # 影像 resize 成 256x256
    transforms.CenterCrop(224), # 切出中央 224x224
    transforms.ToTensor(), # 由 Image 改成 Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Standardized as Normal(mean,std)
])


<hr>
<h3>Image 讀取測試影像＆顯示</h3>
<p>Image Pre-processing 後成為 Tensor</p>

In [ ]:
# Image 讀取測試影像＆顯示

import cv2
from PIL import Image
import matplotlib.pyplot as plt

img = Image.open('pineapple.jpg')
plt.imshow(img)
plt.show()

img_tensor = preprocess(img)

print(img_tensor.shape)


<hr>
<h3>OpenCV 讀取測試影像</h3>
<p>Image Pre-processing 後成為 Tensor</p>

In [ ]:
import cv2
import numpy as np

img_cv2 = cv2.imread('pineapple.jpg')

cv2.imshow('VGG16', img_cv2)
cv2.waitKey(5000)
cv2.destroyAllWindows()

# OpenCV to Image 格式轉換
img_cv2 = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB)
img = Image.fromarray(img_cv2)

img_tensor = preprocess(img)

print(img_tensor.shape)


<hr>
<h3>準備好 VGG16 所需的 Tensor 格式（n, 3, 224, 224）</h3>
<p>n 是影像數（目前 n = 1）</p>

In [ ]:
# 準備好 VGG16 所需的 Tensor 格式（n, 3, 224, 224）

import torch

x = torch.zeros(1, 3, 224, 224)

x[0] = img_tensor

print(x.shape)


<hr>
<h3>VGG16 辨識</h3>

In [ ]:
# VGG16 辨識

import torch

output = vgg16(x)

lst = output[0].tolist()
idx = lst.index(max(lst))

print('class index = ', idx)
print('class name  = ', class_names[idx])


<hr>
<h3 style="color:orange">VGG16 影像辨識（整合版）</h3>

In [ ]:
# VGG16 影像辨識（整合版）

import torch
from torchvision import transforms
import cv2
from PIL import Image
import matplotlib.pyplot as plt

# 載入 CIFAR-1000 物體類別名稱

with open('imagenet_classes.txt', 'r') as fp:
    txt = fp.read()
fp.close()
class_names = txt.split('\n')

# 載入 VGG16 Model

# 下載
vgg16 = torch.hub.load('pytorch/vision:v0.6.0', 'vgg16', pretrained=True)

# ANN 網路模型執行（原定義啟用）
vgg16.eval()

# Image Pre-Processing 定義

preprocess = transforms.Compose([
    transforms.Resize(256), # 影像 resize 成 256x256
    transforms.CenterCrop(224), # 切出中央 224x224
    transforms.ToTensor(), # 由 Image 改成 Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Standardized as Normal(mean,std)
])

# Image 讀取測試影像＆顯示

img = Image.open('pineapple.jpg')
# img = Image.open('banana.jpg')
plt.imshow(img)
plt.show()

img_tensor = preprocess(img)

# 準備好 VGG16 所需的 Tensor 格式（n, 3, 224, 224）

x = torch.zeros(1, 3, 224, 224)
x[0] = img_tensor

# VGG16 辨識

output = vgg16(x)

lst = output[0].tolist()
idx = lst.index(max(lst))

print('class index = ', idx)
print('class name  = ', class_names[idx])
